# Facies segmentation Python Demo

This Jupyter notebook demonstrates how to run facies segmentation with seismic data using OpenVINO&trade; Toolkit.

This model is used for seismic interpretation tasks. Facies is the characteristics of a rock that reflects its origin and differentiates a unit from the others around it.  Mineralogy and sedimentary source, fossil content, sedimentary structures and texture distinguish one facies from another. Data are presented in the 3D arrays.

See the source repository to learn more about the model architecture and training method - https://github.com/yalaudah/facies_classification_benchmark

The `IR` model was created with OpenVINO&trade; [Model Optimizer](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)

To convert the model from ONNX to `IR`, we used the `mo_onnx.py` script with the `--extension` flag as shown below:
 
 ```bash
 python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_onnx.py \
    --input_model model.onnx \
    --extension openvino_pytorch_layers/mo_extensions
```
Where `openvino_pytorch_layers/mo_extensions` is python code from [openvino_pytorch_layers](https://github.com/dkurt/openvino_pytorch_layers) repository

---
**NOTE**

Currently, only 2D visualization is supported in `jupyter lab`, for 3D visualizations please use `jupyter notebook` instead. To switch from Jupyter Lab to classic Jupyter Notebook, select `Help` from the menu bar and then `Launch Classic Notebook`.

---

## Demo Output

The application displays 3D visualization inside the Jupyter notebook using itkwidget with resulting instance classification masks.

## How It Works
At start-up, the demo application loads a network and a given dataset file to Inference Engine plugin. When inference is complete, the application displays the 3D itkwidget viewer with facies interpretation.

## Installation of dependencies

### Setup virtual-env

Step 1: Import the required packages

In [ ]:
import os
import sys
from collections import defaultdict

import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from openvino.inference_engine import IECore
from openvino_extensions import get_extensions_path
from tqdm import tqdm

### Download the model:

Step 1: Create a model directory and download the model:

In [ ]:
! mkdir model
! curl -L -o model/facies-segmentation-deconvnet.bin https://www.dropbox.com/s/x0c7ao8kebxykj1/facies-segmentation-deconvnet.bin?dl=1/
! curl -L -o model/facies-segmentation-deconvnet.xml https://www.dropbox.com/s/g288xdcd7xumqm7/facies-segmentation-deconvnet.xml?dl=1/
! curl -L -o model/facies-segmentation-deconvnet.mapping https://www.dropbox.com/s/a7kge25hfpjnhvf/facies-segmentation-deconvnet.mapping?dl=1/

### Download dataset:

The open source dataset used in this demo is available on GitHub: https://github.com/yalaudah/facies_classification_benchmark

In [ ]:
# Download dataset
! mkdir data
! curl -L -o data/test2_seismic.npy https://www.dropbox.com/s/sbj2atyukpjgssx/test2_seismic.npy?dl=1/

### Define functions

In [ ]:
def get_config():
    config = defaultdict(str)
    config.update(
        {
            "model": os.path.join("model", "facies-segmentation-deconvnet.xml"),
            "data_path": os.path.join("data", "test2_seismic.npy"),
            "name_classes": [
                "upper_ns",
                "middle_ns",
                "lower_ns",
                "rijnland_chalk",
                "scruff",
                "zechstein",
            ],
            "slice_no": 101,
            "slice_axis": 1,
            "edge_one": (0, 30, 0),
            "edge_two": (500, 199, 244),
        }
    )

    return config

In [ ]:
def normalize(data, mu=0, std=1):
    if not isinstance(data, np.ndarray):
        data = np.array(data)
    data = (data - data.flatten().mean()) / data.flatten().std()
    return data * std + mu


def load_data(config):
    data_format = config["data_path"].split(".")[1]
    assert not (
        config["data_path"].split(".")[0] == "" or data_format == ""
    ), f'Invalid path to data file: {config["data_path"]}'
    if data_format == "npy":
        data = np.load(config["data_path"])
    elif data_format == "dat":
        data = np.fromfile(config["data_path"])
    elif data_format == "segy":
        import segyio

        data = segyio.tools.cube(config["data_path"])
        data = np.moveaxis(data, -1, 0)
        data = np.ascontiguousarray(data, "float32")
    else:
        assert False, f"Unsupported data format: {data_format}"

    data = normalize(data, mu=1e-8, std=0.2097654)
    print(f"[INFO] Dataset has been loaded, shape is {data.shape}")
    print(
        f"[INFO] Dataset mean is {data.flatten().mean():.5f}, std {data.flatten().std():.5f}"
    )

    x_min = min(config["edge_one"][0], config["edge_two"][0])
    x_max = max(config["edge_one"][0], config["edge_two"][0])
    y_min = min(config["edge_one"][1], config["edge_two"][1])
    y_max = max(config["edge_one"][1], config["edge_two"][1])
    z_min = min(config["edge_one"][2], config["edge_two"][2])
    z_max = max(config["edge_one"][2], config["edge_two"][2])
    x_lim, y_lim, z_lim = data.shape
    assert x_min >= 0 and y_min >= 0 and z_min >= 0
    assert x_max < x_lim and y_max < y_lim and z_max < z_lim, "Invalid edges"
    sub_data = data[x_min:x_max, y_min:y_max, z_min:z_max]
    return sub_data

In [ ]:
def reshape_model(net, shape, axis=None):
    if axis is None:
        index_of_dim = np.argmin(shape)
    else:
        index_of_dim = axis
    input_data_shape = list(shape)
    del input_data_shape[index_of_dim]

    input_net_info = net.input_info
    input_name = next(iter(input_net_info))
    input_net_shape = input_net_info[input_name].input_data.shape

    print(f"[INFO] Infer should be on {input_data_shape} resolution")
    if input_data_shape != input_net_shape[-2:]:
        net.reshape({input_name: [1, 1, *input_data_shape]})
        print(f"[INFO] Reshaping model to fit for slice shape: {input_data_shape}")
    else:
        print(f"[INFO] Use not reshaped model")

In [ ]:
def infer_cube(exec_net, data, axis=None):
    if axis is None:
        index_of_dim = np.argmin(data.shape)
    else:
        index_of_dim = axis
    predicted_cube = np.empty(data.shape)
    size = data.shape[index_of_dim]
    for slice_index in tqdm(range(size)):
        if index_of_dim == 0:
            inp = data[slice_index, :, :]
            out = exec_net.infer(inputs={"input": inp})["output"]
            out = np.argmax(out, axis=1).squeeze()
            predicted_cube[slice_index, :, :] = out
        if index_of_dim == 1:
            inp = data[:, slice_index, :]
            out = exec_net.infer(inputs={"input": inp})["output"]
            out = np.argmax(out, axis=1).squeeze()
            predicted_cube[:, slice_index, :] = out
        if index_of_dim == 2:
            inp = data[:, :, slice_index]
            out = exec_net.infer(inputs={"input": inp})["output"]
            out = np.argmax(out, axis=1).squeeze()
            predicted_cube[:, :, slice_index] = out
    return predicted_cube

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)


def show_legend(N, cmap_name):
    base = plt.cm.get_cmap(cmap_name)
    color_list = base(np.linspace(0, 1, N))
    print(color_list)


def show_legend(labels, cmap):
    N = len(labels)
    fig = plt.figure(figsize=(12, 6))
    ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
    cb1 = mpl.colorbar.ColorbarBase(
        ax1,
        cmap=cmap,
        ticks=np.arange(0, N, 1) / N + 1 / (2 * N),
        orientation="horizontal",
    )
    cb1.ax.set_xticklabels(labels, fontsize=20)
    cb1.set_label("Legend", fontsize=24)
    plt.show()

### Get config and load dataset

In [ ]:
config = get_config()
data = load_data(config)

## Running

### Load model

In [ ]:
ie = IECore()
ie.add_extension(get_extensions_path(), "CPU")
net = ie.read_network(config["model"])

We use `get_extensions_path` function to   to set up our own CPU extension to infer this model. This is necessary because the model uses the `MaxUnpool2D` layer, which is not yet supported by OpenVINO ™.


### Run model on a single slice

Let's define function for decodig class labels into a color

In [ ]:
LABEL_COLOURS = np.asarray(
    [
        [69, 117, 180],
        [145, 191, 219],
        [224, 243, 248],
        [254, 224, 144],
        [252, 141, 89],
        [215, 48, 39],
    ]
)

In [ ]:
def decode_segmap(label_mask):
    """Decode segmentation class labels into a color image
    Args:
        label_mask (np.ndarray): an (M,N) array of integer values denoting
            the class label at each spatial location.
        plot (bool, optional): whether to show the resulting color image
            in a figure.
    Returns:
        (np.ndarray, optional): the resulting decoded color image.
    """
    r = label_mask.copy()
    g = label_mask.copy()
    b = label_mask.copy()
    for ll in range(0, len(LABEL_COLOURS)):
        r[label_mask == ll] = LABEL_COLOURS[ll, 0]
        g[label_mask == ll] = LABEL_COLOURS[ll, 1]
        b[label_mask == ll] = LABEL_COLOURS[ll, 2]
    rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
    rgb[:, :, 0] = r / 255.0
    rgb[:, :, 1] = g / 255.0
    rgb[:, :, 2] = b / 255.0
    return rgb

Function for plotting output from the model:

In [ ]:
def show_facies_interpretation(input_slice, output_labels_slice):
    from matplotlib.colors import LinearSegmentedColormap

    res_image = decode_segmap(output_labels_slice.squeeze())

    color_list = LABEL_COLOURS / 255
    cm = LinearSegmentedColormap.from_list("custom_cmap", color_list, N=6)

    fig, axs = plt.subplots(1, 2, figsize=(15, 12))
    fig.suptitle("Facies classification results", fontsize=22)

    axs[0].imshow(input_slice, cmap="Greys")
    axs[0].set_title("Input data slice", fontsize=15)

    im = axs[1].imshow(res_image, cmap=cm)
    axs[1].set_title("Interpretation of the slice", fontsize=15)

    cbaxes = fig.add_axes([0.95, 0.15, 0.02, 0.65])
    cb = fig.colorbar(
        im, ax=axs[0], cax=cbaxes, ticks=[0.23, 0.36, 0.5, 0.65, 0.78, 0.93]
    )
    cb.ax.set_yticklabels(
        ["upper_ns", "middle_ns", "lower_ns", "rijnland_chalk", "scruff", "zechstein"],
        fontsize=12,
        ha="left",
    )

#### Prepare slice for 2d visualization

In [ ]:
seismic_data = np.load(config["data_path"])
if config["slice_axis"] == 0:
    inp_slice = seismic_data[config["slice_no"], :, :]
elif config["slice_axis"] == 1:
    inp_slice = seismic_data[:, config["slice_no"], :]
elif config["slice_axis"] == 2:
    inp_slice = seismic_data[:, :, config["slice_no"]]
else:
    assert False, "Invalid slice_axis, it must be int 0, 1 or 2"

seismic_data = normalize(seismic_data, mu=1e-8, std=0.2097654)

#### Prepare model

If you want to run inference on another device, such as a `GPU`, you must specify the `net` variable like the examples below:
* for `GPU`:
```
exec_net = ie.load_network(network=net, device_name="HETERO:GPU, CPU")
```

In [ ]:
reshape_model(net, seismic_data.shape, axis=config["slice_axis"])
exec_net = ie.load_network(network=net, device_name="CPU")

#### Infer model on a slice

In [ ]:
out_slice_from_model = exec_net.infer(inputs={"input": inp_slice})["output"]
out_labels_slice = np.argmax(out_slice_from_model, axis=1).squeeze()

#### Show output

In [ ]:
show_facies_interpretation(inp_slice, out_labels_slice)

**Output explanation**:

Different facies are identified as output classes. Facies represent a body of rock with specific characteristics that differentiate it from others.

More details on the dataset can be found at this link: https://arxiv.org/abs/1901.07659


### Run model cube (multiple slices) for 3D visualization

In [ ]:
is_visualize_3d = False
try:
    from itkwidgets import view

    is_visualize_3d = True
except ModuleNotFoundError:
    print(
        "[WARNING]: itkwidgets is not installed,\n to see 3D visualizations, please install itkwidgets by uncommenting the following cell "
    )

In [ ]:
# %pip install itkwidgets==0.23.1
# from itkwidgets import view
# is_visualize_3d = True

#### Prepare model

In [ ]:
reshape_model(net, data.shape, axis=1)
exec_net = ie.load_network(network=net, device_name="CPU")

#### Run inference (multiple slices)

In [ ]:
predicted_data = infer_cube(exec_net, data, axis=1)

Inference is now running on the model. Slices along axis 1 are fed to the input and the results are combined into an output cube (`predicted_data`).

### Visualize original and predicted data

* Prepare original data viewer

In [ ]:
if is_visualize_3d:
    viewer_orig_data = view(data, shadow=False, annotations=False)
    count_of_greys = 100
    viewer_orig_data.cmap = np.array(
        [
            [i / count_of_greys, i / count_of_greys, i / count_of_greys]
            for i in range(count_of_greys)
        ]
    )

* Prepare predicted data viewer

In [ ]:
if is_visualize_3d:
    cmap = discrete_cmap(len(config["name_classes"]), "jet")
    show_legend(config["name_classes"], cmap)
    viewer_interpret_data = view(predicted_data, shadow=False, annotations=False)
    viewer_interpret_data.cmap = cmap
    widgets.link(
        (viewer_interpret_data, "camera"), (viewer_orig_data, "camera")
    )  # link widget cameras
    pass

* Run render

In [ ]:
is_visualize_3d and viewer_interpret_data or print("3D rendering disabled")

In [ ]:
is_visualize_3d and viewer_orig_data or print("3D rendering disabled")

You can now see a visualization of the interpreted and raw seismic data. It is also possible to use your mouse to interactively rotate, zoom and explore the interpreted data. If you don't see rendering, just restart this jupyter-notebook.


#### Thanks to the authors:

A machine-learning benchmark for facies classification / Yazeed Alaudah,\
Patrycja Michałowicz, Motaz Alfarraj, Ghassan AlRegib // Interpretation. 2019. T. 7, № 3.\
C. SE175-SE187.
